In [1]:
import pandas as pd
import seaborn as sb
import numpy as np
from sklearn.preprocessing import LabelEncoder

import geocoder

pd.options.display.float_format = "{:.2f}".format

In [2]:
#yel2019 = pd.read_csv("/Users/james.fung/Desktop/test_scripts/via/yellow_tripdata_2016-06.csv")
yel2019 = pd.read_csv("/Users/jamesfung/Dropbox/Resumes/Resume/NYC/2020_resume/challenges/via/yellow_tripdata_2016-06.csv")

In [3]:
#zones = pd.read_csv("/Users/james.fung/Desktop/test_scripts/via/taxi_zones.csv")
zones = pd.read_csv("/Users/jamesfung/Dropbox/Resumes/Resume/NYC/2020_resume/challenges/via/taxi+_zone_lookup.csv")

In [5]:
#boros = pd.read_csv("/Users/james.fung/Downloads/nybb (1).csv")
boros = pd.read_csv("/Users/jamesfung/Dropbox/Resumes/Resume/NYC/2020_resume/challenges/via/nybb.csv")

In [64]:
centroids = pd.read_csv("/Users/jamesfung/Dropbox/Resumes/Resume/NYC/2020_resume/challenges/via/NHoodNameCentroids.csv")

In [6]:
yel2019.head(10)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2016-06-09 21:06:36,2016-06-09 21:13:08,2,0.79,-73.98,40.76,1,N,-73.98,40.75,2,6.00,0.50,0.50,0.00,0.00,0.30,7.30
1,2,2016-06-09 21:06:36,2016-06-09 21:35:11,1,5.22,-73.98,40.74,1,N,-73.98,40.67,1,22.00,0.50,0.50,4.00,0.00,0.30,27.30
2,2,2016-06-09 21:06:36,2016-06-09 21:13:10,1,1.26,-73.99,40.75,1,N,-74.00,40.74,1,6.50,0.50,0.50,1.56,0.00,0.30,9.36
3,2,2016-06-09 21:06:36,2016-06-09 21:36:10,1,7.39,-73.98,40.77,1,N,-73.93,40.85,1,26.00,0.50,0.50,1.00,0.00,0.30,28.30
4,2,2016-06-09 21:06:36,2016-06-09 21:23:23,1,3.10,-73.99,40.73,1,N,-73.99,40.77,1,13.50,0.50,0.50,2.96,0.00,0.30,17.76
5,2,2016-06-09 21:06:36,2016-06-09 21:19:21,1,2.17,-74.00,40.74,1,N,-73.99,40.76,1,10.50,0.50,0.50,2.36,0.00,0.30,14.16
6,2,2016-06-09 21:06:36,2016-06-09 21:30:13,5,6.02,-73.98,40.74,1,N,-73.90,40.75,2,21.50,0.50,0.50,0.00,0.00,0.30,22.80
7,1,2016-06-09 21:06:37,2016-06-09 21:16:47,1,1.40,-73.99,40.75,1,N,-73.98,40.76,1,8.50,0.50,0.50,1.95,0.00,0.30,11.75
8,1,2016-06-09 21:06:37,2016-06-09 21:15:44,1,1.20,-73.98,40.78,1,N,-73.97,40.77,2,8.00,0.50,0.50,0.00,0.00,0.30,9.30
9,1,2016-06-09 21:06:37,2016-06-09 21:23:57,2,1.90,-73.98,40.75,1,N,-73.99,40.75,1,12.00,0.50,0.50,3.33,0.00,0.30,16.63


In [7]:
yel2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11135470 entries, 0 to 11135469
Data columns (total 19 columns):
VendorID                 int64
tpep_pickup_datetime     object
tpep_dropoff_datetime    object
passenger_count          int64
trip_distance            float64
pickup_longitude         float64
pickup_latitude          float64
RatecodeID               int64
store_and_fwd_flag       object
dropoff_longitude        float64
dropoff_latitude         float64
payment_type             int64
fare_amount              float64
extra                    float64
mta_tax                  float64
tip_amount               float64
tolls_amount             float64
improvement_surcharge    float64
total_amount             float64
dtypes: float64(12), int64(4), object(3)
memory usage: 1.6+ GB


### Preliminary Feature Extraction

Let's first extract some useful information from tpep_pickup_datetime. Things such as day of the week, day of month, month, and trip duration.

In [8]:
#Create a column for the clean date.
yel2019['clean_pickup_datetime'] = pd.to_datetime(yel2019['tpep_pickup_datetime'], format= '%Y-%m-%d %H:%M:%S')
yel2019['clean_dropoff_datetime'] = pd.to_datetime(yel2019['tpep_dropoff_datetime'], format= '%Y-%m-%d %H:%M:%S')

In [9]:
#Extract day of the week, day of month, and month.
pickup = yel2019['clean_pickup_datetime']
dropoff = yel2019['clean_dropoff_datetime']
yel2019['month'] = pickup.dt.month
yel2019['dom'] = pickup.dt.day
yel2019['dow'] = pickup.dt.dayofweek

In [10]:
#Extract the duration of the trip.
yel2019['diff_sec'] = (dropoff-pickup)/np.timedelta64(1,'s')

In [11]:
#Drop the unecessary columns.
yel2019 = yel2019.drop(['tpep_pickup_datetime','tpep_dropoff_datetime'],axis=1)

In [12]:
yel2019.head(10)

,VendorID,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,...,tip_amount,tolls_amount,improvement_surcharge,total_amount,clean_pickup_datetime,clean_dropoff_datetime,month,dom,dow,diff_sec
0,2,2,0.79,-73.98,40.76,1,N,-73.98,40.75,2,...,0.00,0.00,0.30,7.30,2016-06-09 21:06:36,2016-06-09 21:13:08,6,9,3,392.00
1,2,1,5.22,-73.98,40.74,1,N,-73.98,40.67,1,...,4.00,0.00,0.30,27.30,2016-06-09 21:06:36,2016-06-09 21:35:11,6,9,3,1715.00
2,2,1,1.26,-73.99,40.75,1,N,-74.00,40.74,1,...,1.56,0.00,0.30,9.36,2016-06-09 21:06:36,2016-06-09 21:13:10,6,9,3,394.00
3,2,1,7.39,-73.98,40.77,1,N,-73.93,40.85,1,...,1.00,0.00,0.30,28.30,2016-06-09 21:06:36,2016-06-09 21:36:10,6,9,3,1774.00
4,2,1,3.10,-73.99,40.73,1,N,-73.99,40.77,1,...,2.96,0.00,0.30,17.76,2016-06-09 21:06:36,2016-06-09 21:23:23,6,9,3,1007.00
5,2,1,2.17,-74.00,40.74,1,N,-73.99,40.76,1,...,2.36,0.00,0.30,14.16,2016-06-09 21:06:36,2016-06-09 21:19:21,6,9,3,765.00
6,2,5,6.02,-73.98,40.74,1,N,-73.90,40.75,2,...,0.00,0.00,0.30,22.80,2016-06-09 21:06:36,2016-06-09 21:30:13,6,9,3,1417.00
7,1,1,1.40,-73.99,40.75,1,N,-73.98,40.76,1,...,1.95,0.00,0.30,11.75,2016-06-09 21:06:37,2016-06-09 21:16:47,6,9,3,610.00
8,1,1,1.20,-73.98,40.78,1,N,-73.97,40.77,2,...,0.00,0.00,0.30,9.30,2016-06-09 21:06:37,2016-06-09 21:15:44,6,9,3,547.00
9,1,2,1.90,-73.98,40.75,1,N,-73.99,40.75,1,...,3.33,0.00,0.30,16.63,2016-06-09 21:06:37,2016-06-09 21:23:57,6,9,3,1040.00


### Converting Lat/Long Into Meaningful Features

In [29]:
locations = yel2019[['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude',]]

In [30]:
locations.describe()

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
count,11135470.00,11135470.00,11135470.00,11135470.00
mean,-73.05,40.24,-73.12,40.28
std,8.21,4.52,7.88,4.34
min,-118.19,0.00,-118.19,0.00
25%,-73.99,40.74,-73.99,40.73
50%,-73.98,40.75,-73.98,40.75
75%,-73.97,40.77,-73.96,40.77
max,0.00,64.10,106.25,60.04


We are only interested in Manhattan's efficiency, so let's remove trips that are outside of Manhattan.

In [43]:
yel2019 = yel2019[
    (yel2019.pickup_longitude >= -74.2563) & (yel2019.pickup_longitude <= -73.6543) &
    (yel2019.pickup_latitude >= 40.4941) & (yel2019.pickup_latitude <= 40.9313) &
    (yel2019.dropoff_longitude >= -74.2563) & (yel2019.dropoff_longitude <= -73.6543) &
    (yel2019.dropoff_latitude >= 40.4941) & (yel2019.dropoff_latitude <= 40.9313)]

In [45]:
yel2019.shape

(10972566, 23)

In [52]:
[str(yel2019.pickup_latitude) + ',' + str(yel2019.pickup_longitude)]

['0          40.76\n1          40.74\n2          40.75\n3          40.77\n4          40.73\n5          40.74\n6          40.74\n7          40.75\n8          40.78\n9          40.75\n10         40.74\n11         40.76\n12         40.79\n13         40.72\n14         40.75\n15         40.75\n16         40.74\n17         40.79\n18         40.76\n19         40.76\n20         40.71\n21         40.74\n22         40.74\n23         40.75\n24         40.78\n25         40.77\n26         40.78\n27         40.77\n28         40.76\n29         40.77\n            ... \n11135439   40.73\n11135440   40.75\n11135441   40.76\n11135443   40.73\n11135444   40.73\n11135445   40.75\n11135446   40.77\n11135447   40.69\n11135448   40.71\n11135449   40.75\n11135450   40.75\n11135451   40.76\n11135452   40.73\n11135453   40.77\n11135454   40.77\n11135455   40.67\n11135456   40.87\n11135457   40.71\n11135458   40.78\n11135459   40.74\n11135460   40.76\n11135461   40.76\n11135462   40.71\n11135463   40.75\n11135464

Let's now convert these lat/longs to the borooughs using a geocoder.

In [47]:
from geopy.geocoders import Nominatim
geolocator = Nominatim()

borough = []
loc = ['40.4941, -74.2563']
for l in loc:
    sub = str(geolocator.reverse(l))
    borough.append(sub.split(', ')[2])
borough

/Users/jamesfung/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  


['Richmond County']

In [65]:
centroids.latitude.head(10)

0   40.89
1   40.87
2   40.89
3   40.90
4   40.89
5   40.88
6   40.88
7   40.90
8   40.88
9   40.88
Name: latitude, dtype: float64

In [56]:
from math import radians, cos, sin, asin, sqrt
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    # Radius of earth in kilometers is 6371
    km = 6371* c
    return km

### Preliminary Data Cleaning

In [ ]:
#Drop some unecesssary columns.
taxi_merged = taxi_merged.drop(['VendorID',
                                'PULocationID',
                                'DOLocationID',
                                'clean_pickup_datetime',
                                'clean_dropoff_datetime'],axis=1)

In [ ]:
#Convert some columns into factors.
factors = ['RatecodeID','payment_type','month','dom','dow']
for i in factors:
    taxi_merged[i] = taxi_merged[i].astype(object)

In [ ]:
#Check out the numerical columns.
pd.options.display.float_format = "{:.2f}".format
taxi_merged.describe()

There are some oddities in this data. Let's do some cleanup.

In [ ]:
taxi_merged = taxi_merged[taxi_merged['month']==1]
taxi_merged = taxi_merged[taxi_merged['passenger_count']>0]
taxi_merged = taxi_merged[taxi_merged['fare_amount']>0]
taxi_merged = taxi_merged[taxi_merged['tip_amount']>=0]
taxi_merged = taxi_merged[taxi_merged['diff_sec']>0]

In [ ]:
taxi_merged.describe()

Looks a lot better except for the extreme outliers. Let's remove all rows that are greater than the 99th percentile.

In [ ]:
#This is lazy coding - sorry!
numerical_cols = taxi_merged.describe().columns
for cols in numerical_cols:
    taxi_merged = taxi_merged[taxi_merged[cols]<=taxi_merged[cols].quantile(q=.99)]

In [ ]:
taxi_merged.describe()

In [ ]:
#Any missing values?
taxi_merged.isnull().sum()

### Preliminary EDA

Goal: Propose a metric or algorithm to assess the potential efficiency of aggregating rides 
from many vehicles into one, given the available data. Make realistic assumptions and any necessary simplifications and state them.

In [ ]:
taxi_merged.head(10)

One of the first thoughts that comes to mind are single unit passengers who hail a taxi in the same time and same location of another single unit passenger.

In [ ]:
#What is the distribution of passenger counts?
sb.countplot(taxi_merged['passenger_count'])

A majority of taxi rides are single passenger rides.

In [ ]:
taxi_merged['DOZone'].nunique()